In [ ]:
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from threading import Thread
import multiprocessing as mp
from joblib import Parallel, delayed
max_cpu = mp.cpu_count()
print(f'{max_cpu} cpu\'s detected')
import numpy as np


from math import sqrt
from time import sleep, time

2 cpu's detected


# Global Interpreter Lock
Please keep in mind the Python Global Interpreter Lock (GIL): No two lines of Pyhton code can execute at the same time (ever)

In [ ]:
def poolworker(i, comp_cost_per_job):
    data = np.random.uniform(size=(1024, 1024))
    for i in range(comp_cost_per_job*30):
        np.matmul(data, data) # already done in parallel in by Numpy, use other function
    return i

In [ ]:
def poolworker(i, comp_cost_per_job):
    sleep(comp_cost_per_job)
    return 

# Using (built-in) Concurrent futures

In [ ]:
n_jobs = 5
comp_cost_per_job = 1
t0 = time()
results = [poolworker(i, comp_cost_per_job) for i in range(n_jobs)]
print("Sequential", time() - t0)

# Thread Pool
t0 = time()
pool = ThreadPoolExecutor(8) # can crash when using interactively
futures = [pool.submit(poolworker, i, comp_cost_per_job) for i in range(n_jobs)]
results = [future.result() for future in futures]
print("Concurrent Thread Pool", time() - t0)

# Process Pool
t0 = time()
pool = ProcessPoolExecutor(8) # can crash when using interactively
futures = [pool.submit(poolworker, i, comp_cost_per_job) for i in range(n_jobs)]
results = [future.result() for future in futures]
print("Concurrent Process Pool", time() - t0)

Here Thread Pools don't give any speedup, whereas Process Pools do

Please Keep in mind that Process Pools use different address spaces, you can't use any variables defined outside the function's scope, whereas with Thread Pools, this is possible! :)

# multiprocessing package

In [ ]:
# n_jobs = 100
# comp_cost_per_job = 10
# Sequential
t0 = time()
results = [poolworker(i, comp_cost_per_job) for i in range(n_jobs)]
print("Sequential", time() - t0)


t0 = time()
# Step 1: Init multiprocessing.Pool()
pool = mp.Pool(mp.cpu_count()) # can crash when using interactively
# Step 2: `pool.apply` the `howmany_within_range()`
results = [pool.apply(poolworker, args=(i, comp_cost_per_job)) for i in range(n_jobs)]
# Step 3: Don't forget to close
pool.close()    
print("MP Process Pool", round(time() - t0, 1))

# Joblib

In [ ]:
n_jobs = 10
comp_cost_per_job = 4

# Sequential
t0 = time()
results = [poolworker(i, comp_cost_per_job) for i in range(n_jobs)]
print("Sequential", time() - t0)

# Joblib
t0 = time()
backend = 'multiprocessing' # can crash when using interactively
results = Parallel(n_jobs=-1, backend=backend, batch_size=1)(delayed(
    poolworker)(i, comp_cost_per_job) for i in range(n_jobs))
print(f"Joblib {backend} Pool", round(time() - t0, 1))

# Joblib
t0 = time()
backend = 'loky' # so far robust when using interactively
results = Parallel(n_jobs=-1, backend=backend)(delayed(
    poolworker)(i, comp_cost_per_job) for i in range(n_jobs))
print(f"Joblib {backend} Pool", round(time() - t0, 1))

Sequential 37.586479902267456


# Dask

In [ ]:
from dask.distributed import Client
import bokeh

if 'client' not in globals():
    client = Client(processes=True, n_workers=3)  # start local workers as processes
# or
# clientthr = Client(processes=False)  # start local workers as threads

/usr/local/lib/python3.6/dist-packages/distributed/bokeh/core.py:57: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


In [ ]:
n_jobs = 10
comp_cost_per_job = 4
# sequestial
t0 = time()
results = [poolworker(i, comp_cost_per_job) for i in range(n_jobs)]
print("  Sequential",round(time() - t0, 1))
# Dask futures
# so far robust when using interactively
t0 = time() 
# results = Parallel(n_jobs=-1, backend=backend)(delayed(
#     poolworker)(i, comp_cost_per_job) for i in range(n_jobs))
futures = [client.submit(poolworker, i, comp_cost_per_job) for i in range(n_jobs)]
results = client.gather(futures)
print("Dask Process Pool", round(time() - t0, 1))

  Sequential 40.0


distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/distributed/protocol/core.py", line 114, in loads
    header = msgpack.loads(header, use_list=False, **msgpack_opts)
  File "msgpack/_unpacker.pyx", line 195, in msgpack._cmsgpack.unpackb
ValueError: tuple is not allowed for map key
distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/distributed/protocol/core.py", line 114, in loads
    header = msgpack.loads(header, use_list=False, **msgpack_opts)
  File "msgpack/_unpacker.pyx", line 195, in msgpack._cmsgpack.unpackb
ValueError: tuple is not allowed for map key
distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/distributed/protocol/core.py", line 114, in loads
    header = msgpack.loads(header, use_list=False, **ms

ValueError: ignored

In [ ]:
import sys, os
RGCPD_path = '/Users/semvijverberg/surfdrive/Scripts/RGCPD/RGCPD'
if RGCPD_path not in sys.path:
    sys.path.append(RGCPD_path)

import core_pp, functions_pp
tmp = functions_pp.time_mean_periods
filename = '/Users/semvijverberg/surfdrive/ERA5/input_raw/preprocessed/sst_1979-2020_jan_dec_monthly_1.0deg.nc'
filename = '/Users/semvijverberg/surfdrive/ERA5/input_raw/preprocessed/sst_1979-2018_1jan_31dec_daily_1.0deg.nc'
ds = core_pp.import_ds_lazy(filename)

# get_subdates(pd.to_datetime(ds.time.values), start_end_date, start_end_year=None, lpyr=False,
#                  returngroups=False)
def aggregate():
    ds = core_pp.import_ds_lazy(filename)
    ds_aggr = tmp(ds, np.array([['12-01', '02-28']]),
                                range(1980,2018))
n_jobs = 2

# t0 = time() 
# [aggregate(ds) for i in range(n_jobs)]
# print("  Sequential",round(time() - t0, 1))

t0 = time() 
client.upload_file(os.path.join(RGCPD_path,'core_pp.py'))## Send local package to workers
client.upload_file(os.path.join(RGCPD_path,'functions_pp.py'))


import dask

futures = [dask.delayed(aggregate)() for i in range(n_jobs)]
%timeit dask.compute(*futures)
print("Dask Process Pool", round(time() - t0, 1))

# %timeit [aggregate(ds) for i in range(n_jobs)]

# %timeit futures = [client.submit(aggregate, ds) for i in range(n_jobs)]

distributed.scheduler - ERROR - Couldn't gather keys {'aggregate-15935078-61b9-45d0-bcc9-674c34ecfa9f': ['tcp://127.0.0.1:56571']} state: ['memory'] workers: []
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:56571'], aggregate-15935078-61b9-45d0-bcc9-674c34ecfa9f
NoneType: None
distributed.client - WARNING - Couldn't gather 1 keys, rescheduling {'aggregate-15935078-61b9-45d0-bcc9-674c34ecfa9f': ('tcp://127.0.0.1:56571',)}
distributed.scheduler - ERROR - Couldn't gather keys {'aggregate-15935078-61b9-45d0-bcc9-674c34ecfa9f': ['tcp://127.0.0.1:56571']} state: ['memory'] workers: []
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:56571'], aggregate-15935078-61b9-45d0-bcc9-674c34ecfa9f
NoneType: None
distributed.client - WARNING - Couldn't gather 1 keys, rescheduling {'aggregate-15935078-61b9-45d0-bcc9-674c34ecfa9f': ('tcp://127.0.0.1:56571',)}
distributed.nanny - WARNING - Restarting worke

KeyboardInterrupt: 

distributed.nanny - WARNING - Restarting worker


In [ ]:
futures


[Delayed('aggregate-05ac5e08-372a-4015-84c1-cd14af1378b9'),
 Delayed('aggregate-912ee12e-dadf-4230-aa55-93628693c492')]

(None, None)

(Delayed('aggregate-4df5caf8-fd73-49af-943c-bb648b5e51e5'),
 Delayed('aggregate-a54fde81-4f87-47fb-84b2-82779df058bd'))